In [1]:
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)
    #print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)
    #print(np.expand_dims(train_data, axis=1).shape)
    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

    return train_data, train_label, test_data, test_label

In [2]:
train_x,train_y,test_x,test_y = read_bci_data()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [3]:

class EEGNet(nn.Module):
    def __init__(self, activation='relu', dropout=0.25):
        super(EEGNet, self).__init__()
        #super().__init__()

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU()
        else:
            self.activation = nn.ELU(alpha=1.0)

        self.conv2d = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        self.depthwiseConv2d = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            self.activation,
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0),
            nn.Dropout(p=dropout)
        )
        self.separableConv2d = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            self.activation,
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),
            nn.Dropout(p=dropout)
        )
        self.classify = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=736, out_features=2, bias=True)
        )

    def forward(self, x):
        out = self.conv2d(x)
        out = self.depthwiseConv2d(out)
        out = self.separableConv2d(out)
        out = self.classify(out)
        #out = self.classify(out.flatten(start_dim=1))
        return out

In [4]:
class DeepConvNet(nn.Module):
    def __init__(self, activation='relu'):
        #super().__init__()
        super(DeepConvNet, self).__init__()
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU()
        else:
            self.activation = nn.ELU(alpha=1.0)

        self.layer0 = nn.Sequential(
            nn.Conv2d(1, 25, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.Conv2d(25, 25, kernel_size=(2, 1), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(25),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer1 = nn.Sequential(
            nn.Conv2d(25, 50, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(50),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer2 = nn.Sequential(
            nn.Conv2d(50, 100, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(100),
            self.activation,
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5))

        self.layer3 = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0), bias=False),
            nn.BatchNorm2d(200),
            self.activation)

        self.layer4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(1, 2)),
            nn.Dropout(p=0.5),
            nn.Flatten(),
            nn.Linear(in_features=43*200, out_features=2, bias=True))

    def forward(self, x):
        out = self.layer0(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

In [5]:
def train(model, device, train_loader, optimizer, criterion, learning_rate, epoch,step_size, gamma, log_period):
    
    opt = optimizer(model.parameters(), lr=learning_rate)
    scheduler = StepLR(opt, step_size=step_size, gamma=gamma)
    loss_list = []
    
    for i in range(epoch):
        loss_sum = 0
        model.train()
        for data,target in train_loader:
            data, target = data.to(device), target.to(device)
            target = target.to(torch.int64)
            opt.zero_grad()
            out = model(data)
            loss = criterion(model(data),target)
            loss_sum += loss
            loss_list.append(loss)
            loss.backward()
            opt.step()
            
        scheduler.step()
        if((i+1) % log_period == 0):
            print(f"Train epoch: {i+1}   loss: {loss_sum/(log_period*len(train_loader))}")
            
def test(model, device, test_loader):
    model.eval()

In [6]:
def main():
    use_cuda = torch.cuda.is_available()
    
    device = torch.device("cuda" if use_cuda else "cpu")
    batch_size = 10
    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': batch_size}
    if use_cuda:
        print("Use GPU for training...")
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)
    else:
        print("Use CPU to training...")
        
    train_x,train_y,test_x,test_y = read_bci_data()
    train_dataset = TensorDataset(Tensor(train_x), Tensor(train_y))
    test_dataset = TensorDataset(Tensor(test_x), Tensor(test_y))
    train_loader = DataLoader(train_dataset, **train_kwargs)
    
    model = EEGNet().to(device)
    #model = DeepConvNet().to(device)
    train(model=model, 
          device=device, 
          train_loader=train_loader, 
          optimizer=optim.Adam, 
          criterion=nn.CrossEntropyLoss(),
          learning_rate=1e-2, 
          epoch=100, 
          step_size=1, 
          gamma=0.7, 
          log_period=10 )
    

main()

Use GPU for training...
(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)
Train epoch: 10   loss: 0.03443295881152153
Train epoch: 20   loss: 0.03345677629113197
Train epoch: 30   loss: 0.031335510313510895
Train epoch: 40   loss: 0.033834103494882584
Train epoch: 50   loss: 0.03278464078903198
Train epoch: 60   loss: 0.03151490166783333
Train epoch: 70   loss: 0.03264815732836723
Train epoch: 80   loss: 0.03410971164703369
Train epoch: 90   loss: 0.03173312917351723
Train epoch: 100   loss: 0.03257611766457558
